<a href="https://colab.research.google.com/github/SinaQP/Brief_Burst/blob/main/Brief_Burst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install openai langchain langchain-community langchain-openai faiss-cpu --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.2/473.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [18]:
!pip install docx2txt python-docx --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.3 MB/s eta 0:00:00


In [3]:
import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

class AvalaiClient:
    def __init__(self, api_key="", base_url="", model_name="gpt-4o-mini"):
        self.api_key, self.base_url = self._set_api_config(api_key, base_url)
        self.model_name = model_name
        self.client = openai.OpenAI(api_key=self.api_key, base_url=self.base_url)

    def get_chat_model(self):
        return ChatOpenAI(model_name=self.model_name, openai_api_key=self.api_key, openai_api_base=self.base_url)

    def get_embeddings(self):
        return OpenAIEmbeddings(openai_api_key=self.api_key, openai_api_base=self.base_url)

    def _set_api_config(self, api_key, base_url):
        default_api_key = ""
        default_base_url = "https://api.avalai.ir/v1"
        return api_key or default_api_key, base_url or default_base_url



In [5]:
avalai_client = AvalaiClient()
llm = avalai_client.get_chat_model()

In [12]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader('./The Evolution of Job Search Technology.docx')
documents = loader.load()

[Document(metadata={'source': './The Evolution of Job Search Technology.docx'}, page_content='The Evolution of Job Search Technology\n\n\n\nIntroduction: The Dawn of Employment Exploration\n\n\n\n\nThe quest for meaningful work has been a cornerstone of human existence, evolving alongside societal and technological advancements. In the earliest days, job searches were informal—word of mouth in tight-knit communities or apprenticeships arranged through familial ties. However, as populations grew and industries emerged, the need for structured job-finding mechanisms became evident. Fast forward to the 21st century, and the landscape of job searching has transformed dramatically, driven by technological innovation. This document explores the evolution of job search technology, from rudimentary postings to sophisticated artificial intelligence systems, and speculates on the future of employment discovery.\n\n\n\nThe Pre-Digital Era: Newspapers and Noticeboards\n\n\n\n\nBefore the internet,

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=40)
texts = text_splitter.split_documents(documents)

In [23]:
# saving the Cunks
# from docx import Document

# doc = Document()

# for i, chunk in enumerate(texts, 1):
#     doc.add_paragraph(f"Chunk {i}:")
#     doc.add_paragraph(chunk.page_content)
#     doc.add_paragraph("")

# doc.save("Split_Evolution_of_Job_Search_Technology.docx")

In [35]:
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
map_prompt_template = """
Shorten the following text while keeping its original wording and main ideas intact. Remove extra details but don’t rephrase completely:
{text}
"""
reduce_prompt_template = """
Combine these shortened texts into a concise summary, keeping the original phrasing where possible:
{text}
"""
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])
reduce_prompt = PromptTemplate(template=reduce_prompt_template, input_variables=["text"])
chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=reduce_prompt
)
response = chain.invoke({"input_documents": texts})

In [36]:
from docx import Document
summary = response["output_text"]

doc = Document()
doc.add_heading("Summary of The Evolution of Job Search Technology", level=1)
doc.add_paragraph(summary)

# Save the document
output_file = "Summary_Evolution_of_Job_Search_Technology.docx"
doc.save(output_file)

'The article explores the evolution of job search technology from traditional methods to advanced digital platforms. It details the transition from print media and community noticeboards to online job boards and social media, emphasizing the role of technology in reshaping recruitment practices. The rise of artificial intelligence and algorithms in the hiring process is highlighted, showcasing how these innovations offer efficiency but also raise ethical concerns. The future of job discovery is envisioned with hyper-personalized experiences, virtual reality interactions, and AI-driven career guidance, while also addressing challenges like data privacy and the digital divide. Ultimately, the quest for meaningful work remains a constant amidst technological advancements.'